In [149]:
import re
import os
import json
import statistics
import datetime
import warnings
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
import polars as pl
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [151]:
df = pl.read_parquet(os.path.join("data","cnb_vyber.parquet"))
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('pocet_stran'))

In [155]:
rocni_pocty = df.filter(pl.col('rok').is_between(1901, 2023)).group_by('rok').len().sort('rok').rename({'len':'knih'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
rocni_pocty

rok,knih
datetime[μs],u32
1901-01-01 00:00:00,1807
1902-01-01 00:00:00,2038
1903-01-01 00:00:00,1973
1904-01-01 00:00:00,1896
1905-01-01 00:00:00,1811
…,…
2019-01-01 00:00:00,13726
2020-01-01 00:00:00,12474
2021-01-01 00:00:00,12935


In [165]:
rocni_strany = df.filter(pl.col('rok').is_between(1901, 2023)).group_by('rok').agg(pl.col("pocet_stran").sum()).sort('rok').rename({'pocet_stran':'stran'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
rocni_strany

rok,stran
datetime[μs],i64
1901-01-01 00:00:00,223084
1902-01-01 00:00:00,274893
1903-01-01 00:00:00,235235
1904-01-01 00:00:00,218549
1905-01-01 00:00:00,223786
…,…
2019-01-01 00:00:00,2631646
2020-01-01 00:00:00,2399513
2021-01-01 00:00:00,2504423


In [285]:
novinky = df.filter(pl.col('rok').is_between(1901, 2023)).filter(pl.col('pocet_stran') >= 50).sort('rok').unique(subset=["245_a","100_a"])
novinky = novinky.group_by('rok').len().with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
novinky = novinky.sort('rok').rename({'len':'novinky'})

In [707]:
puvodni = df.filter(pl.col('rok').is_between(1901, 2023)).explode('041_a').filter(pl.col('041_a') == 'cze').group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'originaly'}).sort('rok')
puvodni

rok,originaly
i64,u32
1901,275
1902,314
1903,305
1904,301
1905,308
…,…
2019,5112
2020,4525
2021,4465


In [709]:
nepuvodni = df.filter(pl.col('rok').is_between(1901, 2023)).explode('041_a').filter(pl.col('041_a') != 'cze').group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'preklady'}).sort('rok')
nepuvodni

rok,preklady
i64,u32
1901,34
1902,54
1903,55
1904,48
1905,42
…,…
2019,739
2020,614
2021,678


In [721]:
preklady = puvodni.join(nepuvodni, on='rok', how='left')
preklady = preklady.with_columns((pl.col('originaly') + pl.col('preklady')).alias('celkem'))
preklady = preklady.with_columns((pl.col('preklady') / pl.col('celkem')).alias('podíl překladů'))
preklady = preklady.select(pl.col(['rok','podíl překladů']))
preklady = preklady.with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
preklady

rok,podíl překladů
datetime[μs],f64
1901-01-01 00:00:00,0.110032
1902-01-01 00:00:00,0.146739
1903-01-01 00:00:00,0.152778
1904-01-01 00:00:00,0.137536
1905-01-01 00:00:00,0.12
…,…
2019-01-01 00:00:00,0.126303
2020-01-01 00:00:00,0.119478
2021-01-01 00:00:00,0.13183


In [727]:
do_grafu = rocni_pocty.join(rocni_strany, on="rok", how="left")
do_grafu = do_grafu.join(novinky, on="rok", how="left")
do_grafu = do_grafu.with_columns((pl.col("stran") / (60 * 8 * 365.25)).alias("let čtení"))
do_grafu = do_grafu.join(preklady, on='rok', how='left')
do_grafu = do_grafu.melt(id_vars="rok", value_vars=['knih','stran','let čtení','novinky','podíl překladů'])
do_grafu = do_grafu.filter(pl.col('variable').is_in(['knih','novinky','podíl překladů']))
do_grafu = do_grafu.with_columns(pl.col('variable').replace_strict({'knih': 'všechny publikace', 'novinky': 'novinky nad 50 stran', 'podíl překladů':'podíl překladů'}))
do_grafu

C:\Users\micha\AppData\Local\Temp\ipykernel_18080\4164877985.py:5: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  do_grafu = do_grafu.melt(id_vars="rok", value_vars=['knih','stran','let čtení','novinky','podíl překladů'])


rok,variable,value
datetime[μs],str,f64
1901-01-01 00:00:00,"""všechny publikace""",1807.0
1902-01-01 00:00:00,"""všechny publikace""",2038.0
1903-01-01 00:00:00,"""všechny publikace""",1973.0
1904-01-01 00:00:00,"""všechny publikace""",1896.0
1905-01-01 00:00:00,"""všechny publikace""",1811.0
…,…,…
2019-01-01 00:00:00,"""podíl překladů""",0.126303
2020-01-01 00:00:00,"""podíl překladů""",0.119478
2021-01-01 00:00:00,"""podíl překladů""",0.13183


In [729]:
do_grafu

rok,variable,value
datetime[μs],str,f64
1901-01-01 00:00:00,"""všechny publikace""",1807.0
1902-01-01 00:00:00,"""všechny publikace""",2038.0
1903-01-01 00:00:00,"""všechny publikace""",1973.0
1904-01-01 00:00:00,"""všechny publikace""",1896.0
1905-01-01 00:00:00,"""všechny publikace""",1811.0
…,…,…
2019-01-01 00:00:00,"""podíl překladů""",0.126303
2020-01-01 00:00:00,"""podíl překladů""",0.119478
2021-01-01 00:00:00,"""podíl překladů""",0.13183


In [739]:
def irozhlas_conf():

    return {
        "config" : {
             "title": {'font': "Noticia Text",
                      'fontSize': 14,
                      'anchor': 'start',
                       'fontWeight': 'bolder',
                    'subtitleFont': 'Noticia Text',
                       'subtitleLineHeight': 18,
                      'subtitleFontSize': 12,
                       'subtitleFontWeight': 'normal',
                      'subtitlePadding': 10,
                       'dy': -12,
                      },
             "axis": {
                "labelFont": "Asap",
                "titleFont": "Asap",
                 "fontWeight": "lighter",
                "titleFontWeight": "lighter",
                "labelFontSize": 10,
                "titleFontSize": 10,
                'labelPadding': 2,
                 'titlePadding': 10
            },
            "legend": {
                "labelFont": "Asap",
                "labelFontWeight": "normal",
                "titleFont": "Asap",
                "titleFontWeight": "normal",
                "labelFontSize": 10,
                "titleFontSize": 10
            }
        }
    }

alt.themes.register('irozhlas', irozhlas_conf)
alt.themes.enable('irozhlas')

ThemeRegistry.enable('irozhlas')

In [351]:
titulek = "Vývoj knižní produkce na českém území"
podtitulek = ["Celkové počty záznamů v České národní bibliografii.","Vedle klasických knih může jít i o katalogy výstav apod."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [741]:
alt.Chart(rocni_pocty.to_pandas(), title={'text': titulek, 'subtitle': podtitulek}).mark_bar(color="#7CA6E1").encode(
    alt.X('rok:T', title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Y('knih:Q', title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    color=alt.condition(
        'datum.knih > 15000',  # Using datum to reference the field value
        alt.value('#84c0e4'),
        alt.value('#687fa9')
    )).properties(width=400, height=200)

alt.Chart(...)

In [191]:
do_grafu

rok,variable,value
datetime[μs],str,i64
1901-01-01 00:00:00,"""knih""",1807
1902-01-01 00:00:00,"""knih""",2038
1903-01-01 00:00:00,"""knih""",1973
1904-01-01 00:00:00,"""knih""",1896
1905-01-01 00:00:00,"""knih""",1811
…,…,…
2019-01-01 00:00:00,"""stran""",2631646
2020-01-01 00:00:00,"""stran""",2399513
2021-01-01 00:00:00,"""stran""",2504423


In [227]:
def ceska_cisla(value):
    if abs(value) >= 1_000_000:
        return f"{value/1_000_000:.0f} mil."
    elif abs(value) >= 1_000:
        return f"{value/1_000:.0f} tis."
    else:
        return str(int(value))

In [235]:
ceska_cisla(1000)

'1 tis.'

In [743]:
produkce_faceted = alt.Chart(do_grafu.to_pandas(), title={'text': titulek, 'subtitle': podtitulek}, width=300).mark_area().encode(
    alt.X("rok:T", 
          title=None,
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
    ),
    alt.Y('value:Q', 
          axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6', format='~s'), 
          title=None
    ),
    alt.Color("variable:N", 
              scale=alt.Scale(range=['#8eba21', '#f77961']),
              title=None, 
              legend=None
    ),
    row=alt.Row(
        "variable:N", 
        sort=['všechny publikace', 'novinky nad 50 stran'],
        title=None, 
        spacing=15,
        header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='middle', labelFont='Asap')
    )
).resolve_scale(y='independent').properties(height=90, width=300)

produkce_faceted

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [745]:
def me_to_neurazi(zaklad_grafu, soubor, kredity):

    os.makedirs("grafy", exist_ok=True)
    
    credits = pl.DataFrame({'text': [kredity]})
    credits = alt.Chart(credits.to_pandas(), width=300).encode(text=alt.Text('text:N')).mark_text(fontSize=10,font="Asap", baseline='line-bottom')
    combined_chart = alt.vconcat(zaklad_grafu, credits)

    combined_chart.save(os.path.join("grafy",f"{soubor}.svg"))
    combined_chart.save(os.path.join("grafy",f"{soubor}.png"), ppi=400)
    
    return combined_chart

In [747]:
me_to_neurazi(produkce_faceted, 'produkce', "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.

alt.VConcatChart(...)